In [1]:
import pandas as pd

In [2]:
babylon = pd.read_csv('babylonbee_urls.csv', quotechar="\"")

In [3]:
onion = pd.read_csv('onion_urls_brief.csv')

In [4]:
onion = onion.drop_duplicates()

In [5]:
spoof_2020 = pd.read_csv('spoof_urls_2020.csv')

In [6]:
spoof_2019 = pd.read_csv('spoof_urls_2019.csv')

In [7]:
spoof_2021 = pd.read_csv('spoof_urls_2021.csv')

In [8]:
satire = pd.concat([babylon, onion, spoof_2020, spoof_2021, spoof_2019])

In [9]:
wel = pd.read_csv('../data/WELFake_Dataset.csv')

In [10]:
wel = wel.dropna()

In [11]:
fake_wel = wel[wel['label'] == 1]
fake_wel.drop_duplicates(inplace=True)
real_wel = wel[wel['label'] == 0]
real_wel.drop_duplicates(inplace=True)
satire.drop_duplicates(inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [12]:
real_wel.rename(columns={'title' : 'Heading', 'text' : 'Body'}, inplace=True)
fake_wel.rename(columns={'title' : 'Heading', 'text' : 'Body'}, inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [13]:
real_wel.dropna(inplace=True)
fake_wel.dropna(inplace=True)
satire.dropna(inplace=True)
fake_wel['body_len'] = fake_wel['Body'].apply(lambda x: len(x))
real_wel['body_len'] = real_wel['Body'].apply(lambda x: len(x))
satire['body_len'] = satire['Body'].apply(lambda x: len(x))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

In [14]:
satire = satire[satire['body_len'] > 1]
fake_wel = fake_wel[fake_wel['body_len'] > 1]
real_wel = real_wel[real_wel['body_len'] > 1]

In [15]:
satire['label'] = 'satire'

In [16]:
real_wel['label'] = 'real'

In [17]:
fake_wel['label'] = 'fake'

In [18]:
import fasttext
PRETRAINED_MODEL_PATH = 'lid.176.ftz'
model = fasttext.load_model(PRETRAINED_MODEL_PATH)
def find_lang(t):
    clean = t.replace("\n", " ")
    return model.predict(clean)[0][0]
satire['language'] = satire['Body'].apply(lambda x: find_lang(x))
real_wel['language'] = real_wel['Body'].apply(lambda x: find_lang(x))
fake_wel['language'] = fake_wel['Body'].apply(lambda x: find_lang(x))

In [19]:
satire = satire[satire['language'] == '__label__en']
real_wel = real_wel[real_wel['language'] == '__label__en']
fake_wel = fake_wel[fake_wel['language'] == '__label__en']

In [20]:
satire_final = satire[['Heading', 'Body', 'label']]
satire_final.dropna(inplace=True)

fake_wel = fake_wel.sample(n=len(satire), random_state=1123456)
fake_final = fake_wel[['Heading', 'Body', 'label']]
fake_final.dropna(inplace=True)

real_wel = real_wel.sample(n=len(satire), random_state=1123456)
real_final = real_wel[['Heading', 'Body', 'label']]
real_final.dropna(inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [21]:
final_data = pd.concat([satire_final, fake_final, real_final])
final_data.dropna(inplace=True)

In [22]:
len(final_data[final_data['label'] == 'fake'])
#len(final_data[final_data['label'] == 'real'])
#len(final_data[final_data['label'] == 'satire'])

19925

In [23]:
final_data['Heading_Body'] = final_data[['Heading', 'Body']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [24]:
final_data

,Heading,Body,label,Heading_Body
0,Here Are 10 Babylon Bee Jokes Explained (Just ...,From the Atlantic to the Pacific a lot of peop...,satire,Here Are 10 Babylon Bee Jokes Explained (Just ...
1,Pete Buttigieg Says He Cannot Come Into Work A...,WASHINGTON D.C.—With catastrophic labor shorta...,satire,Pete Buttigieg Says He Cannot Come Into Work A...
2,"Instead Of Kryptonite, New LGBTQ+ Superman Wil...",BURBANK CA—The brilliant and courageous writer...,satire,"Instead Of Kryptonite, New LGBTQ+ Superman Wil..."
3,Bernie Retires As His Vision Of Making The U.S...,BURLINGTON VT—Bernie Sanders has retired as a ...,satire,Bernie Retires As His Vision Of Making The U.S...
4,Katie Couric Admits She Edited Interview To Re...,U.S.—Katie Couric has admitted to editing an i...,satire,Katie Couric Admits She Edited Interview To Re...
...,...,...,...,...
15517,"Raising 5 Sons Alone, 3 With Autism, Takes a T...",Though Melissa Ferrer moved into her new home ...,real,"Raising 5 Sons Alone, 3 With Autism, Takes a T..."
39730,Man Shot After Striking Off-Duty New York Dete...,A man wielding an meat cleaver near Pennsylv...,real,Man Shot After Striking Off-Duty New York Dete...
40378,Trump to say in security speech that China is ...,WASHINGTON (Reuters) - President Donald Trump ...,real,Trump to say in security speech that China is ...
15732,Exclusive: As Democratic attorneys general tar...,(Reuters) - For years the national political o...,real,Exclusive: As Democratic attorneys general tar...


In [25]:
import re
# Convert to lowercase
clean_final_data = final_data.applymap(lambda s: s.lower())

# Fix Contractions
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
contraction_dict_2 = {"ain’t": "is not", "aren’t": "are not","can’t": "cannot", "’cause": "because", "could’ve": "could have", "couldn’t": "could not", "didn’t": "did not",  "doesn’t": "does not", "don’t": "do not", "hadn’t": "had not", "hasn’t": "has not", "haven’t": "have not", "he’d": "he would","he’ll": "he will", "he’s": "he is", "how’d": "how did", "how’d’y": "how do you", "how’ll": "how will", "how’s": "how is",  "I’d": "I would", "I’d’ve": "I would have", "I’ll": "I will", "I’ll’ve": "I will have","I’m": "I am", "I’ve": "I have", "i’d": "i would", "i’d’ve": "i would have", "i’ll": "i will",  "i’ll've": "i will have","i’m": "i am", "i’ve": "i have", "isn’t": "is not", "it’d": "it would", "it’d’ve": "it would have", "it’ll": "it will", "it’ll’ve": "it will have","it’s": "it is", "let’s": "let us", "ma’am": "madam", "mayn’t": "may not", "might’ve": "might have","mightn’t": "might not","mightn’t’ve": "might not have", "must’ve": "must have", "mustn’t": "must not", "mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have","o’clock": "of the clock", "oughtn’t": "ought not", "oughtn’t’ve": "ought not have", "shan’t": "shall not", "sha’n’t": "shall not", "shan’t’ve": "shall not have", "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will", "she’ll’ve": "she will have", "she’s": "she is", "should’ve": "should have", "shouldn’t": "should not", "shouldn’t’ve": "should not have", "so’ve": "so have","so’s": "so as", "this’s": "this is","that’d": "that would", "that’d’ve": "that would have", "that’s": "that is", "there’d": "there would", "there’d’ve": "there would have", "there’s": "there is", "here’s": "here is","they’d": "they would", "they’d’ve": "they would have", "they’ll": "they will", "they’ll’ve": "they will have", "they’re": "they are", "they’ve": "they have", "to’ve": "to have", "wasn’t": "was not", "we’d": "we would", "we’d’ve": "we would have", "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are", "we’ve": "we have", "weren’t": "were not", "what’ll": "what will", "what’ll’ve": "what will have", "what’re": "what are",  "what’s": "what is", "what’ve": "what have", "when’s": "when is", "when’ve": "when have", "where’d": "where did", "where’s": "where is", "where’ve": "where have", "who’ll": "who will", "who’ll’ve": "who will have", "who’s": "who is", "who’ve": "who have", "why’s": "why is", "why’ve": "why have", "will’ve": "will have", "won’t": "will not", "won’t’ve": "will not have", "would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have", "y’all": "you all", "y’all’d": "you all would","y’all’d’ve": "you all would have","y’all’re": "you all are","y’all’ve": "you all have","you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will", "you’ll’ve": "you will have", "you’re": "you are", "you’ve": "you have"}

def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)
contractions_2, contractions_re_2 = _get_contractions(contraction_dict_2)

def replace_contractions(text):
    def replace_1(match):
        return contractions[match.group(0)]
    def replace_2(match):
        return contractions_2[match.group(0)]
    clean_1 = contractions_re.sub(replace_1, text)
    clean_2 = contractions_re_2.sub(replace_2, clean_1)
    return clean_2
clean_final_data['Heading_Body'] = clean_final_data['Heading_Body'].apply(replace_contractions)
clean_final_data['Heading'] = clean_final_data['Heading'].apply(replace_contractions)
clean_final_data['Body'] = clean_final_data['Body'].apply(replace_contractions)

import string
# Remove Puncuation
def remove_punc(s):
    #punc_dict = {'!': ' ! ', '"': ' " ', '#': ' # ', '$': ' $ ', '%': ' % ', '&': ' & ', "'": " ' ", '(': ' ( ', ')': ' ) ', '*': ' * ', '+': ' + ', ',': ' , ', '-': ' - ', '.': ' . ', '/': ' / ', ':': ' : ', ';': ' ; ', '<': ' less than ', '=': ' equals ', '>': ' greater than ', '?': ' ? ', '@': ' @ ', '[': ' [ ', '\\': ' \\ ', ']': ' ] ', '^': ' ^ ', '_': ' _ ', '`': ' ` ', '{': ' { ', '|': ' | ', '}': ' } ', '~': ' ~ ', '—':' — '}
    punc_dict = {key: ' ' for key in string.punctuation}
    punc_dict['“'] = None
    punc_dict['—'] = ' '
    punc_dict['’'] = ' '
    table = str.maketrans(punc_dict)  # OR {key: None for key in string.punctuation}
    new_s = s.str.translate(table)
    return new_s

clean_final_data = clean_final_data.apply(remove_punc, axis=1)

clean_final_data['Heading_Body'] = clean_final_data['Heading_Body'].apply(lambda x: re.sub("(babylon bee)|(onion)|(spoof)|(satire)|(reuters)", " ", x))
clean_final_data['Heading'] = clean_final_data['Heading'].apply(lambda x: re.sub("(babylon bee)|(onion)|(spoof)|(satire)|(reuters)", " ", x))
clean_final_data['Body'] = clean_final_data['Body'].apply(lambda x: re.sub("(babylon bee)|(onion)|(spoof)|(satire)|(reuters)", " ", x))


In [26]:
# Remove extraneous HTML spaces
clean_final_data['Heading_Body'] = clean_final_data['Heading_Body'].apply(lambda x: str(x).replace(u'\xa0', u' '))
clean_final_data['Heading'] = clean_final_data['Heading'].apply(lambda x: str(x).replace(u'\xa0', u' '))
clean_final_data['Body'] = clean_final_data['Body'].apply(lambda x: str(x).replace(u'\xa0', u' '))

In [27]:
# Remove extra white spaces between words
clean_final_data['Heading_Body'] = clean_final_data['Heading_Body'].apply(lambda x: re.sub("\s\s+", " ", x))
clean_final_data['Heading'] = clean_final_data['Heading'].apply(lambda x: re.sub("\s\s+", " ", x))
clean_final_data['Body'] = clean_final_data['Body'].apply(lambda x: re.sub("\s\s+", " ", x))

In [28]:
clean_final_data

,Heading,Body,label,Heading_Body
0,here are 10 jokes explained just in case you d...,from the atlantic to the pacific a lot of peop...,satire,here are 10 jokes explained just in case you d...
1,pete buttigieg says he cannot come into work a...,washington d c with catastrophic labor shortag...,satire,pete buttigieg says he cannot come into work a...
2,instead of kryptonite new lgbtq superman will ...,burbank ca the brilliant and courageous writer...,satire,instead of kryptonite new lgbtq superman will ...
3,bernie retires as his vision of making the u s...,burlington vt bernie sanders has retired as a ...,satire,bernie retires as his vision of making the u s...
4,katie couric admits she edited interview to re...,u s katie couric has admitted to editing an in...,satire,katie couric admits she edited interview to re...
...,...,...,...,...
15517,raising 5 sons alone 3 with autism takes a tol...,though melissa ferrer moved into her new home ...,real,raising 5 sons alone 3 with autism takes a tol...
39730,man shot after striking off duty new york dete...,a man wielding an meat cleaver near pennsylvan...,real,man shot after striking off duty new york dete...
40378,trump to say in security speech that china is ...,washington president donald trump will lay out...,real,trump to say in security speech that china is ...
15732,exclusive as democratic attorneys general targ...,for years the national political organization...,real,exclusive as democratic attorneys general targ...


In [29]:
from collections import Counter
# Create Vocabulary
counter = Counter()
for comment in clean_final_data.Heading_Body:
    counter.update(comment.split())

counter

Counter({'here': 20869,
         'are': 137215,
         '10': 9701,
         'jokes': 557,
         'explained': 2736,
         'just': 46937,
         'in': 514803,
         'case': 11434,
         'you': 96587,
         'do': 53383,
         'not': 167801,
         'get': 27325,
         'them': 38187,
         'from': 116668,
         'the': 1520216,
         'atlantic': 866,
         'to': 778284,
         'pacific': 1436,
         'a': 622705,
         'lot': 8221,
         'of': 683092,
         'people': 62123,
         'all': 65348,
         'across': 8045,
         'country': 23633,
         'understand': 3917,
         'that': 376147,
         'is': 318538,
         'because': 30773,
         'we': 100467,
         'conservative': 6921,
         'christians': 1821,
         'bad': 6208,
         'at': 124126,
         'humor': 422,
         'nowhere': 592,
         'near': 4878,
         'as': 167100,
         'funny': 1197,
         'leftist': 1151,
         'humorists': 7,

In [30]:
clean_final_data

,Heading,Body,label,Heading_Body
0,here are 10 jokes explained just in case you d...,from the atlantic to the pacific a lot of peop...,satire,here are 10 jokes explained just in case you d...
1,pete buttigieg says he cannot come into work a...,washington d c with catastrophic labor shortag...,satire,pete buttigieg says he cannot come into work a...
2,instead of kryptonite new lgbtq superman will ...,burbank ca the brilliant and courageous writer...,satire,instead of kryptonite new lgbtq superman will ...
3,bernie retires as his vision of making the u s...,burlington vt bernie sanders has retired as a ...,satire,bernie retires as his vision of making the u s...
4,katie couric admits she edited interview to re...,u s katie couric has admitted to editing an in...,satire,katie couric admits she edited interview to re...
...,...,...,...,...
15517,raising 5 sons alone 3 with autism takes a tol...,though melissa ferrer moved into her new home ...,real,raising 5 sons alone 3 with autism takes a tol...
39730,man shot after striking off duty new york dete...,a man wielding an meat cleaver near pennsylvan...,real,man shot after striking off duty new york dete...
40378,trump to say in security speech that china is ...,washington president donald trump will lay out...,real,trump to say in security speech that china is ...
15732,exclusive as democratic attorneys general targ...,for years the national political organization...,real,exclusive as democratic attorneys general targ...


In [31]:
clean_final_data.to_csv('../data/clean_final_data.csv', index=False)